# Atividade 01 - Criando objetos MNE

Dennis Felipe Urtubia e Pedro Perozin

# Carregamento dos dados, labels e descritor

In [1]:
import numpy as np
import json
import mne
import scipy
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

data = np.load('files/data.npy')
labels = np.load('files/labels.npy')

desc_file = open('files/descriptor.json')
descriptor = json.loads(desc_file.read())
desc_file.close()

print('Estruturas => dados', data.shape, 'labels', labels.shape)
print(labels)

Estruturas => dados (125, 257, 1205) labels (125,)
[4 2 3 5 1 2 5 4 2 3 1 5 4 3 2 4 1 2 5 3 4 1 3 1 3 4 2 3 5 1 2 5 4 2 3 1 5
 4 3 2 4 1 2 5 3 4 1 3 1 3 4 2 3 5 1 2 5 4 2 3 1 5 4 3 2 4 1 2 5 3 4 1 3 1
 3 4 2 3 5 1 2 5 4 2 3 1 5 4 3 2 4 1 2 5 3 4 1 3 1 3 4 2 3 5 1 2 5 4 2 3 1
 5 4 3 2 4 1 2 5 3 4 1 3 1 3]


# Criação do EpochsArray

In [2]:
data = data[:,:256,:]

trial_duration = 5
sampling_frequency = data.shape[-1] / trial_duration
montage = mne.channels.make_standard_montage('EGI_256')
ch_names = data.shape[1]
ch_types = 'eeg'

# primeiramente devemos criar o objeto info
info = mne.create_info(montage.ch_names, sampling_frequency, ch_types)

#set experiments montage
info.set_montage(montage)

# por fim a criação do EpochsArray
events = np.array([[index, 0, event] for index, event in enumerate(labels)])
#objeto MNE epoch
epoch = mne.EpochsArray(data, info, events)

print()

125 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped



# Plot de gráficos

In [ ]:
# plotando o padrão EGI 256
print('EGI 256')
montage.plot()

# Filtrando apenas dados dos eletrodos O1, Oz e O2
filtered_epoch = epoch.copy().pick_channels(['E116', 'E126', 'E150','E106','E107','E108','E109','E119','E140','E151','E160','E169'])

filtered_epoch.filter(l_freq = 5.0, h_freq = 14.0)

e = '4'

for i in range(2):
    filtered_epoch[e][i].plot_psd(fmin=5., fmax=14.)

matplotlib.rcParams['figure.figsize'] = [12., 8.]
filtered_epoch[e].plot_psd(fmin = 5., fmax = 14.)
print()

# Aplicando filtros espaciais (Melhorar descrição)

In [3]:
filtered_epoch = epoch.copy().pick_channels(['E116', 'E126', 'E150','E106','E107','E108','E109','E119','E140','E151','E160','E169'])

filtered_epoch.filter(l_freq = 5.0, h_freq = 14.0)

# CAR
epoch_ref = mne.set_eeg_reference(filtered_epoch, ref_channels=['E116', 'E126', 'E150','E106','E107','E108','E109','E119','E140','E151','E160','E169'])
epoch_ref

# Limitando as frequências
fmin = 5.0
fmax = 14.0

# filtros
epoch_highpass = epoch_ref[0].copy().filter(l_freq=None, h_freq=fmax)
epoch_lowpass = epoch_ref[0].copy().filter(l_freq=fmin, h_freq=None)
epoch_bandpass = epoch_ref[0].copy().filter(l_freq=fmin, h_freq=fmax)
epoch_bandpass2 = epoch_ref[0].copy().filter(l_freq=fmin, h_freq=None)
epoch_bandpass2.filter(l_freq=None, h_freq=fmax)

Setting up band-pass filter from 5 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 399 samples (1.656 sec)

EEG channel type selected for re-referencing
Applying a custom EEG reference.
Setting up low-pass filter at 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 227 samples (0.94

<EpochsArray  |   125 events (all good), 0 - 4.99585 sec, baseline off, ~14.0 MB, data loaded,
 '1': 25
 '2': 25
 '3': 30
 '4': 25
 '5': 20>

In [ ]:
epoch.save('files/ssvep-epo.fif', overwrite=True)

# Transformada

In [4]:
from scipy.signal import stft

data = epoch.get_data()
_, _, w = stft(data, fs=241, nperseg=32, noverlap=16)

# Extração de características

In [5]:
# PSD
W = np.abs(w) ** 2

fmn = np.mean(W, axis=-1)

rss = np.sqrt(np.sum(W, axis=-1))

## Finalizando o vetor de características

In [6]:
# finalizando o exemplo com a junção das duas características criadas
features = list()
for feature in (fmn, rss,):
    feature = feature.transpose(0, 2, 1)
    feature = feature.reshape(feature.shape[0] * feature.shape[1], feature.shape[2])
    features.append(feature)

# vetor de características final
X = np.concatenate(features, axis=-1)
print('Shape dos dados:', X.shape)

Shape dos dados: (2125, 512)


## Adaptação do vetor de labels

In [7]:
y = np.load('files/labels.npy')
print('Shape original dos labels', y.shape)

size = int(X.shape[0] / y.shape[0])
y = np.concatenate([y for i in range(size)])
print('Shape final dos labels', y.shape)

Shape original dos labels (125,)
Shape final dos labels (2125,)


# Classificação

### SVM - Support Vector Machine

In [ ]:
for count in range(50):
    for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
        for gamma in [10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]:
            for C in [0.01, 0.1, 1, 10, 100, 1000]:
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y, train_size=0.7, shuffle=True)
                clf = SVC(kernel=kernel,gamma=0.0001, C=C)
                clf = clf.fit(X_train, y_train)
                res = clf.predict(X_test)
                if ( 100*metrics.accuracy_score(y_test, res) >= 86.0): # 92
                    print(count)
                    print('Kernel:{} | Gamma:{} e C:{} | Accuracy: {:.2f}%'.format(
                        kernel, gamma, C, 100*metrics.accuracy_score(y_test, res)))